In [1]:
!pwd

/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/cooc


In [3]:
import glob
import datetime
import glob
import stanza
import pandas as pd
import time
import sys

In [4]:
def filetime(file):
    year=file.split('/')[-1][5:9]
    month=file.split('/')[-1][10:12]
    day=file.split('/')[-1][13:15]
    hour=file.split('/')[-1][16:18]
    return [int(each) for each in [year, month, day, hour]]

In [5]:
def onlywithneighbours(ofthislist):
    try:
        filtered = [each for each in ofthislist 
                    if each+1 in ofthislist or each-1 in ofthislist]
    except TypeError:
        filtered = [each for each in ofthislist 
                    if str(int(each)+1) in ofthislist or str(int(each)-1) in ofthislist]
    return filtered

In [6]:
def split_into_numerical_sequences(inlist):

    inlist=sorted(inlist)

    breakindeces=[i for i,j in enumerate(inlist)
                    if (j+1 not in inlist and j in inlist)]

    sublists=[]
    for index, each in enumerate(breakindeces):
        if index==0:
            sublists.append([x for x in inlist
                               if x<=inlist[each]])
        if index!=0:
            sublists.append([x for x in inlist
                               if x<=inlist[each] and x>inlist[breakindeces[index-1]]])

    return sublists

In [7]:
def validalias(alias):
    if len(str(alias).strip()) > 5 and len(str(alias).strip().split(' ')) > 1: return True
    else: return False

In [8]:
def matchfinder(text,searchforthese):
    matches=[alias
             for persondata_searchtarget in searchforthese
             for alias in persondata_searchtarget
             if validalias(alias) and alias.lower() in str(text).lower()]
    return matches

In [9]:
def fillextracols(whichdictionary, whichrowtofill, withthislist, unlessitslongerthanthis):
    if not len(withthislist) > unlessitslongerthanthis and len(withthislist)>0:
        for i, e in enumerate(withthislist):
            targetdf.loc[whichrowtofill,whichdictionary+str(i)]=e

In [10]:
def prepare_extra_columns(num_of_columns_for_each_dict):
    for each in num_of_columns_for_each_dict:
        for index in range(num_of_columns_for_each_dict[each]):
            targetdf[each + str(index)]=''

In [11]:
class dictionary_class:
    def __init__(self, name, maxcolnum, searchlist=None, geo=False):
        self.name = name
        self.maxcolnum = maxcolnum
        self.searchlist = searchlist
        self.geo = geo

In [12]:
def searchlist_maker(csv,excelfile=False,multisheet=False,columnsfiltered=False, headerwechoose='name_list',**kwargs):
    
    if not multisheet:
        if excelfile: persondatadict = pd.read_excel(csv)

        else:
            persondatadict = pd.read_csv(csv)

        if 'alias_separator' in kwargs:
            persondata_searchlist = [[eachalias.strip()
                                    for eachalias in eachperson.split(kwargs['alias_separator'])]
                                    for eachperson in persondatadict[headerwechoose]
                                    if type(eachperson) != float]
        else:
            persondata_searchlist = [[each]
                                    for each in persondatadict[headerwechoose]]

    if multisheet:
        
        if not columnsfiltered:

            xls=pd.ExcelFile(csv)
            dfs=[pd.read_excel(xls,sheet) for i, sheet in enumerate(xls.sheet_names)]
            persondata_searchlist=[[name] for df in dfs for name in df['name_list'] if type(name) is not float]

        if columnsfiltered:
            
            unifieddf=pd.concat([pd.read_excel(csv, sheetname)[column]
                        for sheetname in pd.ExcelFile(csv).sheet_names
                        for column in pd.read_excel(csv, sheetname)
                        if 'LOOKFOR' in column]).dropna().drop_duplicates()
            
            persondata_searchlist = [[each] for each in unifieddf]
            
    return persondata_searchlist

In [13]:
dictionaries=[
dictionary_class('szotar_2.1',
           10,
           searchlist_maker('/mnt/volume/jupyter/szokereso/szotar_2.1.xlsx',multisheet=True))]

In [14]:
def get_files_sorted_by_date_after_a_date(look_for_this_pattern,cutoffdate):
    
    csvs = glob.glob(look_for_this_pattern)
    datetimes=[datetime.datetime(int(each.split('/')[-1][5:9]),
                             int(each.split('/')[-1][10:12]),
                             int(each.split('/')[-1][13:15]),
                             int(each.split('/')[-1][16:18]),
                             int(each.split('/')[-1][19:21]),
                             int(each.split('/')[-1][22:24])) for each in csvs]

    dt_csvs_filtered=[[dt, csv] for dt, csv in zip(datetimes,csvs) if dt > datetime.datetime(*cutoffdate)]
    
    sorted_filtered_csvs = [csv
                        for _, csv in sorted(
                                         zip([eachpair[0] for eachpair in dt_csvs_filtered],
                                             [eachpair[1] for eachpair in dt_csvs_filtered]))]
    
    return sorted_filtered_csvs

In [ ]:
debugmode = False
wildcard = '/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv'

todofiles=get_files_sorted_by_date_after_a_date('/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv',
                                               [2020,7,1,0])
while len(todofiles) != 0:
    print('processing: '+todofiles[0])    
    
    targetcsv=todofiles[0]
    try:
        targetdf = pd.read_csv(targetcsv)

        prepare_extra_columns({dictionary.name: dictionary.maxcolnum for dictionary in dictionaries})

        cells = list(targetdf['TEXT'])

        start_time = time.time()
        for idictionary, dictionary in enumerate(dictionaries):
            if idictionary==7 or not debugmode:
            #if dictionary.name == 'instit_dict':
                for icell, cell in enumerate(cells):
                    if type(cell) is not float:
                        if icell < 10 or not debugmode:
                            if icell%1000==0: print(icell)
                            if dictionary.searchlist is not None:
                                fillextracols(dictionary.name,icell,matchfinder(cell,dictionary.searchlist),dictionary.maxcolnum)
                                #if debugmode: print('hw')
                            if dictionary.searchlist is None:
                                fillextracols(dictionary.name,icell,stanzanamesearch(cell),dictionary.maxcolnum)
        end_time = time.time()
        print("--- %s seconds ---" % (time.time() - start_time))
        if not debugmode: targetdf.to_csv('/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/cooc/'+targetcsv.split('/')[-1].split('.')[0]+'_szokereso_result_szotar2_1_only.csv')
        print(targetcsv)
    except KeyboardInterrupt:
        print('KeyboardInterrupt')
        break
    except:
        the_type, the_value, the_traceback = sys.exc_info()
        outlist = [the_type, the_value, targetcsv, dictionary.name, icell]
        if not debugmode:
            with open('/mnt/volume/jupyter/szokereso/resultfiles/onlymomentum/cooc'+targetcsv.split('/')[-1].split('.')[0]+'_ERRORLOG_szotar2_1_only.txt', 'w') as f:
                for item in outlist:
                    f.write("%s\n" % item)
                
    todofiles=get_files_sorted_by_date_after_a_date('/mnt/volume/anagy/mediascraper/mediaScraper/output/data*csv',
                                                     filetime(todofiles[0])[:3]+[filetime(todofiles[0])[-1]+1]) # hour increased by 1

processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_00:00:24.csv
0
1000
2000
3000
--- 36.17245531082153 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_00:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_02:00:18.csv
0
1000
2000
3000
--- 35.241490840911865 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_02:00:18.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_04:00:22.csv
0
1000
2000
3000
--- 34.15720987319946 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_04:00:22.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_06:00:26.csv
0
1000
2000
3000
--- 37.63998818397522 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_06:00:26.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-01_08:01:18.csv
1000
2000
3000


0
1000
2000
3000
--- 34.61682176589966 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-03_22:01:05.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_00:00:29.csv
0
1000
2000
3000
--- 33.791643142700195 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_00:00:29.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_02:00:24.csv
0
1000
2000
3000
--- 35.341644287109375 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_02:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_04:00:24.csv
0
1000
2000
3000
--- 34.948501110076904 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_04:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-04_06:00:25.csv
0
1000
2000
3000
--- 32.524670124053955 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/da

0
1000
2000
3000
--- 43.60451912879944 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_00:00:40.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_02:00:22.csv
0
1000
2000
3000
--- 41.968406677246094 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_02:00:22.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_04:00:24.csv
0
1000
2000
3000
--- 45.25126361846924 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_04:00:24.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_06:00:28.csv
0
1000
2000
3000
--- 42.67244815826416 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_06:00:28.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-07_08:01:08.csv
0
1000
2000
3000
--- 45.31112837791443 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_

0
1000
2000
3000
--- 50.3970148563385 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_02:00:22.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_04:00:26.csv
0
1000
2000
3000
--- 48.53510117530823 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_04:00:26.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_06:00:31.csv
0
1000
2000
3000
--- 50.104400634765625 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_06:00:31.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_08:01:10.csv
0
1000
2000
3000
--- 47.80427265167236 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_08:01:10.csv
processing: /mnt/volume/anagy/mediascraper/mediaScraper/output/data_2020-07-10_10:01:48.csv
0
1000
2000
3000
--- 51.17629671096802 seconds ---
/mnt/volume/anagy/mediascraper/mediaScraper/output/data_2